# Basic Saits

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar, calc_missing_rate
from benchpots.datasets import preprocess_physionet2012
data = preprocess_physionet2012(subset='set-a',rate=0.1) # Our ecosystem libs will automatically download and extract it
train_X, val_X, test_X = data["train_X"], data["val_X"], data["test_X"]
print(train_X.shape)  # (n_samples, n_steps, n_features)
print(val_X.shape)  # samples (n_samples) in train set and val set are different, but they have the same sequence len (n_steps) and feature dim (n_features)
print(f"We have {calc_missing_rate(train_X):.1%} values missing in train_X")  
train_set = {"X": train_X}  # in training set, simply put the incomplete time series into it
val_set = {
    "X": val_X,
    "X_ori": data["val_X_ori"],  # in validation set, we need ground truth for evaluation and picking the best model checkpoint
}
test_set = {"X": test_X}  # in test set, only give the testing incomplete time series for model to impute
test_X_ori = data["test_X_ori"]  # test_X_ori bears ground truth for evaluation
indicating_mask = np.isnan(test_X) ^ np.isnan(test_X_ori)  # mask indicates the values that are missing in X but not in X_ori, i.e. where the gt values are 

from pypots.imputation import SAITS  # import the model you want to use
from pypots.nn.functional import calc_mae
saits = SAITS(n_steps=train_X.shape[1], n_features=train_X.shape[2], n_layers=2, d_model=256, n_heads=4, d_k=64, d_v=64, d_ffn=128, dropout=0.1, epochs=150)
saits.fit(train_set, val_set)  # train the model on the dataset
imputation = saits.impute(test_set)  # impute the originally-missing values and artificially-missing values
mae = calc_mae(imputation, np.nan_to_num(test_X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)
saits.save("save_it_here/saits_physionet2012.pypots")  # save the model for future use
saits.load("save_it_here/saits_physionet2012.pypots")  # reload the serialized model file for following imputation or training

# Implementation Test

In [14]:
from tools import utilities
import pandas as pd
import sys
import numpy as np
from pygrinder import mcar, calc_missing_rate
from pypots.imputation import SAITS  # import the model you want to use
from pypots.nn.functional import calc_mae

input_dir = "../../data/04_working_files/"
output_dir = "../../data/05_imputed_files/"
data_files, missing_data_files = utilities.get_working_files(dir=input_dir)
data_files = ["../../" + file for file in data_files]
missing_data_files = ["../../" + file for file in missing_data_files]
_, missing_data, prep_method = utilities.load_working_files(data_files, missing_data_files)

sample_key = next(iter(missing_data.keys()))
df_missing = missing_data[sample_key]
print(f"Missing data sample before imputation:\n{df_missing.head()}")

df_missing_val = df_missing.values.reshape(-1,96,10)
train_df = df_missing_val[:int(len(df_missing_val)*0.7)]
val_df_org = df_missing_val[int(len(df_missing_val)*0.7):int(len(df_missing_val)*0.85)]
test_df_org = df_missing_val[int(len(df_missing_val)*0.85):]


val_df = val_df_org.copy()
test_df = test_df_org.copy()
val_df = mcar(val_df, p=0.2)
test_df = mcar(test_df, p=0.2)


train_set = {"X": train_df}
val_set = {
    "X": val_df,
    "X_ori": val_df_org,
}
test_set = {"X": test_df}
test_set_org = test_df_org
indicating_mask = np.isnan(test_df) ^ np.isnan(test_set_org) # mask indicates the values that are missing in X but not in X_ori, i.e. where the gt values are available


Loading missing data file: ../../data/04_working_files/ts_feature_Wind_direction_missing.csv as Wind_direction_missing
Loading missing data file: ../../data/04_working_files/ts_feature_Air_temperature_missing.csv as Air_temperature_missing
Loading missing data file: ../../data/04_working_files/ts_feature_Wind_speed_missing.csv as Wind_speed_missing
Loading missing data file: ../../data/04_working_files/ts_feature_Air_pressure_missing.csv as Air_pressure_missing
Loading missing data file: ../../data/04_working_files/ts_feature_Relative_humidity_missing.csv as Relative_humidity_missing
Missing data sample before imputation:
                            5904_WIEN-HOHE WARTE  5925_WIEN-INNERE STADT  \
Date                                                                       
2008-08-05T00:00:00.000000                   NaN                     NaN   
2008-08-05T01:00:00.000000                   NaN                     NaN   
2008-08-05T02:00:00.000000                   NaN                  

In [15]:

saits = SAITS(n_steps=train_df.shape[1], n_features=train_df.shape[2], n_layers=2, d_model=256, n_heads=4, d_k=64, d_v=64, d_ffn=128, dropout=0.1, epochs=150)
saits.fit(train_set, val_set)  # train the model on the dataset
imputation = saits.impute(test_set)  # impute the originally-missing values and artificially-missing values
mae = calc_mae(imputation, np.nan_to_num(test_set_org), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)
print(f"MAE on the test set: {mae}")
imputed_df = pd.DataFrame(imputation.reshape(-1,10), columns=df_missing.columns)

2025-10-21 11:02:12 [INFO]: No given device, using default device: cuda
2025-10-21 11:02:12 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-10-21 11:02:12 [INFO]: Using customized MAE as the training loss function.
2025-10-21 11:02:12 [INFO]: Using customized MSE as the validation metric function.
2025-10-21 11:02:12 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,333,424
2025-10-21 11:02:13 [INFO]: Epoch 001 - training loss (MAE): 401.3352, validation MSE: 49567.8711
2025-10-21 11:02:14 [INFO]: Epoch 002 - training loss (MAE): 393.5999, validation MSE: 47588.9617
2025-10-21 11:02:14 [INFO]: Epoch 003 - training loss (MAE): 382.8102, validation MSE: 44962.3555
2025-10-21 11:02:15 [INFO]: Epoch 004 - training loss (MAE): 369.7869, validation MSE: 41752.5148
2025-10-21 11:02:15 [INFO]: Epoch 005 - training loss (MAE): 352.7901, validation MSE: 37951.5727
2025-10-21 11:02:16 [INFO]: Epoch 006 - t

MAE on the test set: 83.39582448631471


# Backup of original data manipulation

In [ ]:
# Split the df_missing in 0.7 train, 0.15 val, 0.15 test
df_missing = df_missing.iloc[int(0.5*len(df_missing)):int(0.55*len(df_missing))]  # using a small portion for quick test
n = len(df_missing)
train_df = df_missing.iloc[:int(0.4*n)]
val_df_org = df_missing.iloc[int(0.4*n):int(0.4*n)*2]
test_df_org = df_missing.iloc[int(0.4*n)*2:]

from pygrinder import mcar, calc_missing_rate
# populate val_df and test_df with additional 10% missing values for imputation testing
val_df = val_df_org.copy()
test_df = test_df_org.copy()
val_df = mcar(val_df.values, p=0.1)
test_df = mcar(test_df.values, p=0.1)
import numpy as np
train_set = {"X": train_df.values.reshape(-1,1,train_df.shape[1])}
val_set = {
    "X": val_df.reshape(-1,1,val_df.shape[1]),
    "X_ori": val_df_org.values.reshape(-1,1,val_df_org.shape[1]),
}
test_set = {"X": test_df.reshape(-1,1,test_df.shape[1])}
test_set_org = test_df_org.values.reshape(-1,1,test_df_org.shape[1])
indicating_mask = np.isnan(test_df) ^ np.isnan(test_set_org)

# Test of imputationgap packege for Brits

In [1]:
from tools import utilities
import pandas as pd
import sys
import numpy as np

input_dir = "../../data/04_working_files/"
output_dir = "../../data/05_imputed_files/"
data_files, missing_data_files = utilities.get_working_files(dir=input_dir)
data_files = ["../../" + file for file in data_files]
missing_data_files = ["../../" + file for file in missing_data_files]
data, missing_data, prep_method = utilities.load_working_files(data_files, missing_data_files)

sample_key = next(iter(missing_data.keys()))
df = data[sample_key.split("_missing")[0]]
df_missing = missing_data[sample_key]
print(f"Missing data sample before imputation:\n{df_missing.head()}")

Missing data sample before imputation:
                            Wind_direction  Relative_humidity  Air_pressure  \
Date                                                                          
2008-08-05T00:00:00.000000           286.0               96.0         962.3   
2008-08-05T01:00:00.000000           287.0               96.0         962.5   
2008-08-05T02:00:00.000000           293.0               96.0         962.3   
2008-08-05T03:00:00.000000           323.0               96.0         961.9   
2008-08-05T04:00:00.000000           324.0               96.0         962.2   

                            Air_temperature  Wind_speed  
Date                                                     
2008-08-05T00:00:00.000000           291.15         0.9  
2008-08-05T01:00:00.000000           291.25         0.6  
2008-08-05T02:00:00.000000           290.85         1.4  
2008-08-05T03:00:00.000000           290.65         0.4  
2008-08-05T04:00:00.000000           290.65         1.0  


In [4]:
ts.algorithms_with_families

['DeepLearning.BRITS',
 'DeepLearning.BayOTIDE',
 'DeepLearning.BitGraph',
 'DeepLearning.DeepMVI',
 'DeepLearning.GAIN',
 'DeepLearning.GRIN',
 'DeepLearning.HKMF_T',
 'DeepLearning.MPIN',
 'DeepLearning.MRNN',
 'DeepLearning.MissNet',
 'DeepLearning.PRISTI',
 'LLMs.GPT4TS',
 'LLMs.NuwaTS',
 'MachineLearning.IIM',
 'MachineLearning.MICE',
 'MachineLearning.MissForest',
 'MachineLearning.XGBOOST',
 'MatrixCompletion.CDRec',
 'MatrixCompletion.GROUSE',
 'MatrixCompletion.IterativeSVD',
 'MatrixCompletion.ROSL',
 'MatrixCompletion.SPIRIT',
 'MatrixCompletion.SVT',
 'MatrixCompletion.SoftImpute',
 'MatrixCompletion.TRMF',
 'PatternSearch.DynaMMo',
 'PatternSearch.STMVL',
 'PatternSearch.TKCM',
 'Statistics.Interpolation',
 'Statistics.KNNImpute',
 'Statistics.MeanImpute',
 'Statistics.MeanImputeBySeries',
 'Statistics.MinImpute',
 'Statistics.ZeroImpute']

In [5]:
from imputegap.recovery.manager import TimeSeries
from imputegap.recovery.imputation import Imputation

ts = TimeSeries()
ts_m = TimeSeries()
ts.import_matrix(df.values[::24, :].T)
ts.normalize(normalizer="min_max")
ts_m.import_matrix(df_missing.values[::24, :].T)
ts_m.normalize(normalizer="min_max")
print(ts.data.shape)
ts_m = ts_m.data
imputer = Imputation.MatrixCompletion.TRMF(ts_m)

> logs: normalization (min_max) of the data - runtime: 0.0005 seconds
> logs: normalization (min_max) of the data - runtime: 0.0002 seconds
(5, 3652)


In [6]:
ts_m

array([[0.79444444, 0.8       , 0.89166667, ..., 0.01944444, 0.99722222,
        0.89722222],
       [0.94366197, 0.81690141, 0.88732394, ..., 0.70422535, 0.87323944,
        0.88732394],
       [0.59001783, 0.64349376, 0.61675579, ..., 0.67379679, 0.69518717,
        0.67914439],
       [0.75458716, 0.79357798, 0.75917431, ..., 0.85091743, 0.79587156,
        0.77293578],
       [0.15      , 0.06666667, 0.13333333, ..., 0.23333333, 0.08333333,
        0.21666667]])

In [7]:
# use Ray Tune to fine tune the imputation algorithm
imputer.impute(user_def=False, params={"input_data": ts.data, "optimizer": "ray_tune"})

# compute the imputation metrics with optimized parameter values
imputer.score(ts.data, imputer.recov_data)

# compute the imputation metrics with default parameter values
imputer_def = Imputation.MatrixCompletion.CDRec(ts_m).impute()
imputer_def.score(ts.data, imputer_def.recov_data)

# print the imputation metrics with default and optimized parameter values
ts.print_results(imputer_def.metrics, text="Default values")
ts.print_results(imputer.metrics, text="Optimized values")

# plot the recovered time series
ts.plot(input_data=ts.data, incomp_data=ts_m, recov_data=imputer.recov_data, nbr_series=9, subplot=True, algorithm=imputer.algorithm, save_path="./imputegap_assets/imputation")



(OPTI) optimizer ray_tune has been called with trmf ...



2025-10-27 09:17:08,336	INFO worker.py:1841 -- Started a local Ray instance.
2025-10-27 09:17:11,169	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949



		(OPTI) > Ray Total accessible CPU cores for parallelization: 15.0

		(OPTI) > Ray Total accessible memory for parallelization: 3.05 GB

		(OPTI) > Ray tune max_concurrent_trials 1, for 1 calls and metric RMSE


		(OPTI) > Ray tune - SEARCH SPACE: {'lags': {'grid_search': [[], [1, 2, 3], [1, 5, 10]]}, 'K': <ray.tune.search.sample.Categorical object at 0x7f835f1645d0>, 'lambda_f': {'grid_search': [0.1, 1.0, 10.0]}, 'lambda_x': {'grid_search': [0.1, 1.0, 10.0]}, 'lambda_w': {'grid_search': [0.1, 1.0, 10.0]}, 'eta': {'grid_search': [0.1, 1.0, 5.0]}, 'alpha': {'grid_search': [100.0, 500.0]}, 'max_iter': <ray.tune.search.sample.Categorical object at 0x7f835f164950>}



(objective_wrapper pid=148315) (IMPUTATION) TRMF: Matrix Shape: ( 5 ,  3652 ) for lags  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] , K  6 , lambda_f  0.1  lambda_x 0.1 , lambda_w  0.1 , eta  0.1  alpha 100.0 , and max_iter  100 )...


Trial name,RMSE
objective_wrapper_56444_00000,0.619953
objective_wrapper_56444_00001,0.621098
objective_wrapper_56444_00002,0.620992
objective_wrapper_56444_00003,0.623201
objective_wrapper_56444_00004,0.620446
objective_wrapper_56444_00005,0.621976
objective_wrapper_56444_00006,0.623696
objective_wrapper_56444_00007,0.62167
objective_wrapper_56444_00008,0.624444
objective_wrapper_56444_00009,0.619215


(objective_wrapper pid=148492) (IMPUTATION) TRMF: Matrix Shape: ( 5 ,  3652 ) for lags  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] , K  10 , lambda_f  0.1  lambda_x 0.1 , lambda_w  0.1 , eta  1.0  alpha 100.0 , and max_iter  100 )... [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(objective_wrapper pid=148650) (IMPUTATION) TRMF: Matrix Shape: ( 5 ,  3652 ) for lags  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] , K  20 , lambda_f  0.1  lambda_x 0.1 , lambda_w  0.1 , eta  5.0  alpha 100.0 , and max_iter  100 )... [repeated 2x across cluster]
(objective_wrapper pid=148803) (IMPUTATION) TRMF: Matrix Shape: ( 5 ,  3652 ) for lags  [1, 2, 3] , K  20 , lambda_f  0.1  lambda_x 0.1 , lambda_w  0.1 , eta  0.1  alpha 100.0 , and max_iter  100 )... [repeated 2x across cluster]
(objective_wrapper pid=148954) (IMPUTATION) TRMF: Mat

2025-10-27 09:44:46,260	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/thurin/ray_results/objective_wrapper_2025-10-27_09-17-11' in 0.0799s.
2025-10-27 09:44:46,313	INFO tune.py:1041 -- Total run time: 1655.14 seconds (1653.20 seconds for the tuning loop).



(OPTI) > Ray tune - optimal parameters:
	{'lags': [], 'K': 20, 'lambda_f': 1.0, 'lambda_x': 1.0, 'lambda_w': 10.0, 'eta': 1.0, 'alpha': 100.0, 'max_iter': 100}



> logs: optimization ray tune - Execution Time: 1655.3886 seconds_____

(IMPUTATION) TRMF: Matrix Shape: ( 5 ,  3652 ) for lags  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] , K  20 , lambda_f  1.0  lambda_x 1.0 , lambda_w  10.0 , eta  1.0  alpha 100.0 , and max_iter  100 )...

> logs: imputation trmf - Execution Time: 0.4880 seconds


(IMPUTATION) CDRec
	Matrix: 5, 3652
	truncation rank: 3
	epsilon: 1e-06
	iterations: 100

> logs: imputation cdrec - Execution Time: 0.3275 seconds.

Default values :
RMSE                 = 0.5990526432165061
MAE                  = 0.5061303170672077
MI                   = 0.10963000501280779
CORRELATION          = 0.09821215399353961

Optimized values :
RMSE                 = 0.6225302684671208
MAE                  = 0.5302746051863956
MI                   = 0.10348179689888731
CORRELATION          = -0.0

'./imputegap_assets/imputation/25_10_27_09_44_49_trmf_plot.jpg'

In [29]:
from imputegap.recovery.imputation import Imputation
from imputegap.recovery.manager import TimeSeries
from imputegap.tools import utils

# initialize the time series object
ts = TimeSeries()

# load and normalize the dataset
ts.load_series(utils.search_path("eeg-alcohol"))
ts.normalize(normalizer="z_score")

# contaminate and impute the time series
ts_m = ts.Contamination.mcar(ts.data)
imputer = Imputation.MatrixCompletion.CDRec(ts_m)

# use Ray Tune to fine tune the imputation algorithm
imputer.impute(user_def=False, params={"input_data": ts.data, "optimizer": "ray_tune"})

# compute the imputation metrics with optimized parameter values
imputer.score(ts.data, imputer.recov_data)

# compute the imputation metrics with default parameter values
imputer_def = Imputation.MatrixCompletion.CDRec(ts_m).impute()
imputer_def.score(ts.data, imputer_def.recov_data)

# print the imputation metrics with default and optimized parameter values
ts.print_results(imputer_def.metrics, text="Default values")
ts.print_results(imputer.metrics, text="Optimized values")

# plot the recovered time series
ts.plot(input_data=ts.data, incomp_data=ts_m, recov_data=imputer.recov_data, nbr_series=9, subplot=True, algorithm=imputer.algorithm, save_path="./imputegap_assets/imputation")

# save hyperparameters
utils.save_optimization(optimal_params=imputer.parameters, algorithm=imputer.algorithm, dataset="eeg-alcohol", optimizer="ray_tune")


(SYS) The dataset is loaded from /home/thurin/Documents/timeseriesimputation/.venv/lib/python3.11/site-packages/imputegap/datasets/eeg-alcohol.txt

> logs: normalization (z_score) of the data - runtime: 0.0005 seconds

(CONT) missigness pattern: MCAR
	selected series: 1, 6, 10, 13, 17, 26, 37, 41, 45, 53, 57, 59, 62
	percentage of contaminated series: 20.0%
	rate of missing data per series: 20.0%
	block size: 10
	security offset: [0-25]
	seed value: 42

(OPTI) optimizer ray_tune has been called with cdrec ...



2025-10-22 19:20:53,490	INFO worker.py:1841 -- Started a local Ray instance.
2025-10-22 19:20:56,542	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949



		(OPTI) > Ray Total accessible CPU cores for parallelization: 15.0

		(OPTI) > Ray Total accessible memory for parallelization: 2.26 GB

		(OPTI) > Ray tune max_concurrent_trials 1, for 1 calls and metric RMSE


		(OPTI) > Ray tune - SEARCH SPACE: {'rank': {'grid_search': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}, 'eps': <ray.tune.search.sample.Float object at 0x7ff39ff9cf50>, 'iters': {'grid_search': [50, 100, 150]}}



Trial name,RMSE
objective_wrapper_78715_00000,0.54874
objective_wrapper_78715_00001,0.543549
objective_wrapper_78715_00002,0.543543
objective_wrapper_78715_00003,0.40398
objective_wrapper_78715_00004,0.405368
objective_wrapper_78715_00005,0.403958
objective_wrapper_78715_00006,0.352783
objective_wrapper_78715_00007,0.352575
objective_wrapper_78715_00008,0.354158
objective_wrapper_78715_00009,0.309537


2025-10-22 19:23:19,349	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/thurin/ray_results/objective_wrapper_2025-10-22_19-20-56' in 0.0086s.
2025-10-22 19:23:19,358	INFO tune.py:1041 -- Total run time: 142.82 seconds (142.76 seconds for the tuning loop).



(OPTI) > Ray tune - optimal parameters:
	{'rank': 11, 'eps': 2.307158026667674e-05, 'iters': 150}



> logs: optimization ray tune - Execution Time: 142.8435 seconds_____


(IMPUTATION) CDRec
	Matrix: 64, 256
	truncation rank: 11
	epsilon: 2.307158026667674e-05
	iterations: 150

> logs: imputation cdrec - Execution Time: 0.0219 seconds.

(IMPUTATION) CDRec
	Matrix: 64, 256
	truncation rank: 3
	epsilon: 1e-06
	iterations: 100

> logs: imputation cdrec - Execution Time: 0.0035 seconds.

Default values :
RMSE                 = 0.40395406855137334
MAE                  = 0.3116556927747662
MI                   = 0.8410754313179323
CORRELATION          = 0.9127290819984342

Optimized values :
RMSE                 = 0.21652212441449295
MAE                  = 0.16076143610948843
MI                   = 1.215765958315648
CORRELATION          = 0.9756886466794615

plots saved in: ./imputegap_assets/imputation/25_10_22_19_23_22_cdrec_plot.jpg

(SYS) Optimization parameters successfully saved to .